In [1]:
import warnings
import pandas as pd
import numpy as np

pd.options.display.max_columns = None #Полностью отображаем столбцы
warnings.filterwarnings('ignore') # Игнорируем какие-либо ошибки при обработке библиотек

In [2]:
df = pd.read_csv('actions_setka.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1453227 entries, 0 to 1453226
Data columns (total 4 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   session_id   1453223 non-null  float64
 1   user_id      1453224 non-null  object 
 2   action_name  1453224 non-null  object 
 3   action_time  1453225 non-null  object 
dtypes: float64(1), object(3)
memory usage: 44.3+ MB


In [5]:
df.head(10)

,session_id,user_id,action_name,action_time
0,1.0,255323.0,post_view,2025-01-22 23:26:44
1,1.0,255323.0,message,2025-01-22 23:38:42
2,1.0,255323.0,post_view,2025-01-22 23:52:43
3,1.0,255323.0,comment,2025-01-22 23:58:09
4,1.0,255323.0,share,2025-01-22 23:47:47
5,1.0,255323.0,like,2025-01-22 23:41:14
6,1.0,255323.0,message,2025-01-22 23:30:13
7,1.0,255323.0,post_view,2025-01-22 23:48:19
8,1.0,255323.0,profile_view,2025-01-23 00:15:03
9,1.0,255323.0,post_view,2025-01-22 23:51:14


In [7]:
# Проверяем в каких столбцах пропуски и их количество.
df.isna().sum()

session_id     4
user_id        3
action_name    3
action_time    2
dtype: int64

### Выявленные проблемы в данных: 
1. **`session_id` имеет тип `float`**, при этом значения представлены в виде целых чисел (1.0, 2.0 и т.д.).  
    Необходимо привести столбец к целочисленному типу после проверки на наличие некорректных значений.

2. **`user_id` имеет тип `object`**, несмотря на то, что визуально значения выглядят как числовые (например, 12345.0).  
   Требуется проверить наличие нечисловых символов или пропусков, влияющих на тип данных.

3. **`action_time` представлен в формате `object`**, хотя данные визуально соответствуют формату даты и времени (YYYY-MM-DD HH:MM:SS).
   Необходимо проверить корректность формата и привести столбец к типу `datetime`.

4. В датасете **1 453 227 строк**, при этом во всех столбцах присутствуют пропущенные значения.  
   Требуется дополнительно проанализировать характер пропусков и принять решение об их обработке (удаление или заполнение). 

In [10]:
# 1. Преобразуем формат из float в int в столбце session_id
df['session_id'] = df['session_id'].astype('Int64') # Int64 - специальный тип, поддерживающий пропуски

In [9]:
# 2. Ищем строки с буквами
mask_letters = df['user_id'].astype(str).str.contains(r'[A-Za-z]', na=False)
df_bad = df[mask_letters]
print('Строки с буквами в user_id:\n', df_bad.head())

Строки с буквами в user_id:
          session_id user_id action_name          action_time
710672        60374  asddjs      asddjs  2025-01-23 20:30:44
1453223        <NA>     NaN         NaN  2028-01-01 00:00:00
1453224        <NA>     NaN         NaN  2028-01-01 00:00:00
1453225        <NA>     NaN      asddjs                  NaN
1453226        <NA>  asddjs         NaN                  NaN


### Проверка корректности идентификаторов пользователей

В столбце `user_id` обнаружены некорректные значения:  
- Строковые значения (`asddjs`), не соответствуют формату идентификатора пользователя
- Строки с пропущенными значениями сразу в нескольких ключевых колонках
- Записи с аномальными датами (`2028-01-01`)

Данные строки являются техническим шумом и не отражают реальное поведение пользователей.

In [17]:
# Считаем сколько некорректных строк и сравниваем с общим объемом.
bad_count = len(df_bad)
total_count = len(df)

share_bad = bad_count / total_count * 100

print(
    f'Некорректные строки: {bad_count}\n'
    f'Всего строк: {total_count}\n'
    f'Доля некорректных строк: {share_bad:6f}%'
)

Некорректные строки: 5
Всего строк: 1453227
Доля некорректных строк: 0.000344%


В результате проверки выявлено 5 некорректных строк, что составляет около 0.000344% от всего датасета.  
Доля таких записей несущественна и не оказывает влияния на результаты анализа, поэтому данные строки можно удалять.

In [28]:
# Удаляем некорректные строки
df = df[~mask_letters]

In [29]:
# Проверяем
df.isna().sum()

session_id     0
user_id        0
action_name    0
action_time    0
dtype: int64

In [19]:
# Преобразуем user_id к формату Int64
df.user_id = df.user_id.astype(float).astype('Int64')

In [24]:
# 3. Проверяем строки, где год не равен 2025
wrong_year = ~df['action_time'].str.startswith('2025', na=False)
df_wrong_years = df[wrong_year]

# Смотрим результат
print(df_wrong_years.head())
print(f'Всего строк с годами не равным 2025: {len(df_wrong_years)}')


        session_id  user_id action_name          action_time
387865       32987   269659     comment  2028-01-01 00:00:00
926278       78729   264812   post_view  2028-01-01 00:00:00
956486       81264   330217   post_view  2028-01-01 00:00:00
Всего строк с годами не равным 2025: 3


### Проверка корректности годов в action_time

В столбце `action_time` проверены строки, где год отличается от 2025.  
Такой подход позволяет выявлять любые аномальные даты, а не только те, которые мы увидели.  
После выявления аномалий считаю нужным исправить их на 2025 год, так как считаю, что это была опечатка.

In [25]:
# Исправляем года, которые не равны 2025
wrong_year = ~df['action_time'].str.startswith('2025', na=False)
df.loc[wrong_year, 'action_time'] = df.loc[wrong_year, 'action_time'].str.replace(r'^\d{4}', '2025', regex=True)

In [26]:
# Преобразуем столбец в datetime
df['action_time'] = pd.to_datetime(df['action_time'], errors='coerce')

In [27]:
# Проверка результата
print(df['action_time'].min(), df['action_time'].max())
print(f'Кол-во строк с некорректными датами после преобразования: {df['action_time'].isna().sum()}')

2025-01-01 00:00:00 2025-01-31 00:58:57
Кол-во строк с некорректными датами после преобразования: 0


### Исправление аномальных годов и преобразование action_time в datetime

В ходе анализа данных в столбце `action_time` были обнаружены аномальные года, отличные от 2025.  
Это могли быть опечатки или баги данных. Чтобы данные были корректными и пригодными для анализа, мы делаем следующие шаги:  

1. **Находим строки с неправильными годами**
   - Используем метод `.str.startswith('2025', na=False)`, чтобы проверить, начинается ли дата с 2025.
   - Инвертируем с помощью `~`, чтобы получить только строки, где год **не равен 2025**.
2. **Испралвяем год на 2025**
   - Для всех выявленных строк заменяем первые 4 цифры (год) на 2025 с помощью регулярного выражения `r'^\d{4}'`.
   - Такой подход безопасно меняет только год, не трогая месяц, день и время.
3. **Приводим колонку к типу datetime**   
   - `errors='coerce'` гарантирует, что некорректные значения превратятся в `NaT`, чтобы не ломался анализ.
   
**Почему так делаем?**
- Автоматически исправляем любые аномальные годы, не предполагая заранее, какие они могут быть.
- Гарантируем, что даные будут пригодны для расчета метрик по сессиям.
- Обработка пропусков `NaT` позволяет безопасно продолжать работу с датами.
   

In [37]:
# Смотрим какие есть значения с толбце action_name
df['action_name'].value_counts(dropna=False)

action_name
post_view       538093
profile_view    268897
like            215094
comment         161432
subscribe       107917
message         107745
share            54042
asddjs               1
comment              1
Name: count, dtype: int64

### Проверка значений в столбце `action_name`

При анализе значений в столбце `action_name` были выявлены некорректные данные:
1. Значение `asddjs` – явный шум в данных
2. Значения `comment` и `comment ` отличаются только наличием пробела в конце, что приводит к дублированию действия.

In [38]:
# Убираем пробелы
df['action_name'] = df['action_name'].str.strip()

In [40]:
# Удаляем "мусорные" действия (asddjs)
df = df[df['action_name'] != 'asddjs']

In [43]:
# Проверяем
df['action_name'].value_counts()

action_name
post_view       538093
profile_view    268897
like            215094
comment         161433
subscribe       107917
message         107745
share            54042
Name: count, dtype: int64

In [44]:
# Сохраняемый обработанный датасет в новый CSV
df.to_csv('processed_action_setka.csv', index = False)